# Rescue of Legacy Goodreads Data
Goodreads API has been depreciated for [several years](https://help.goodreads.com/s/article/Does-Goodreads-support-the-use-of-APIs)  
Exporting my data to store it elsewhere.

In [74]:
import pandas as pd

In [75]:
goodreads = pd.read_csv('goodreads_library_export.csv').sort_values(by=['Date Added'])

### Trello
Storing data in Markdown format for trello

In [77]:
nested = []
for index, row in goodreads.iterrows():
    # Markdown Format
    md_description = "### Legacy Goodreads Data\n"
    for col_name, value in row.items():
        if not pd.isnull(value):
            md_description += f"- **{col_name}** : {value}\n"
    
    data = {
        'name': row['Title'],
        'desc': md_description
    }
    nested.append(data)

In [72]:
import os
from trello import TrelloClient
from dotenv import load_dotenv
import json

load_dotenv()

# Trello API credentials
API_KEY = os.environ['TRELLO_API_KEY']
API_SECRET = os.environ['TRELLO_OAUTH_SECRET']
TOKEN = os.environ['TRELLO_TOKEN']

# Trello board and list IDs
BOARD_NAME = 'Library'
LIST_NAME = 'Completed'

client = TrelloClient(
    api_key=API_KEY,
    api_secret=API_SECRET,
    token=TOKEN,
)

board = next(x for x in client.list_boards() if x.name == BOARD_NAME)
completed_list = next((lst for lst in board.list_lists() if lst.name == LIST_NAME), None)
if not completed_list: 
    print(f"List '{LIST_NAME}' not found in board")
    exit(1)

completed_books = [{'name': card.name, 'desc': card.description} for card in completed_list.list_cards()]

#### Adding Cards

In [1]:
for book in nested[::-1]:
    print(f"adding {book['name']}")
    new_card = completed_list.add_card(
        name=book['name'],
        desc=book['desc'],
        position='bottom'
    )

#### Getting some metadata from Google Books

In [86]:
import requests

def get_book_info_from_title(title, api_key):
    formatted_title = '+'.join(title.split())
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{formatted_title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        # Parse the response to find the ISBN and the canonical volume link
        items = data.get('items', [])
        if not items:
            print(f"No results found for title: {title}")
            return None, None
        else:
            # Assuming the first result is the desired book
            book = items[0]
            volume_info = book['volumeInfo']
            industry_identifiers = volume_info.get('industryIdentifiers', [])
            isbn_13 = None
            isbn_10 = None
            for identifier in industry_identifiers:
                if identifier['type'] == 'ISBN_13':
                    isbn_13 = identifier['identifier']
                elif identifier['type'] == 'ISBN_10':
                    isbn_10 = identifier['identifier']

            # Use the ISBN-13 if available, else fall back to ISBN-10
            isbn = isbn_13 if isbn_13 else isbn_10
            link_val = volume_info.get('infoLink', '')
            return isbn, link_val
    else:
        print(f"Failed to fetch data for title: {title}")
        return None, None

In [2]:
board = next(x for x in client.list_boards() if x.name == BOARD_NAME)
completed_list = next((lst for lst in board.list_lists() if lst.name == LIST_NAME), None)
if not completed_list: 
    print(f"List '{LIST_NAME}' not found in board")
    exit(1)

for card in completed_list.list_cards():
    isbn, google_books_link = get_book_info_from_title(card.name, api_key)

    # If an ISBN and link were found, update the card's description
    if isbn and google_books_link:
        new_description = f"**ISBN:** {isbn}\n**Google Books Link:** [View Book]({google_books_link})\n\n{card.description}"
        card.set_description(new_description)
        print(f"Updated card '{card.name}' with ISBN and link.")
    else:
        print(f"ISBN/Link not found for '{card.name}'. No updates made.")